<a href="https://colab.research.google.com/github/od41/handson-lm/blob/main/04_Langchain_and_optimisation_techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

/bin/bash: line 1: nvidia-smi: command not found


In [6]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf

--2025-03-08 00:55:34--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf
Resolving huggingface.co (huggingface.co)... 18.238.49.112, 18.238.49.10, 18.238.49.117, ...
Connecting to huggingface.co (huggingface.co)|18.238.49.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/8a83c7fb9049a9b2e92266fa7ad04933bb53aa1e85136b7b30f1b8000ff2edef?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-q4.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-q4.gguf%22%3B&Expires=1741398934&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTM5ODkzNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ4NGNlYWQ3OTlkM2U3YzQ4ZTM4ZWU3NDlmNGM2MDU3Nzc2ZTJlOWUvOGE4M2M3ZmI5MDQ5YTliMmU5MjI2NmZhN2FkMDQ5MzNi

In [7]:
!pip install langchain langchain_community

In [4]:
# command to install on a CPU
%pip install --upgrade --quiet  llama-cpp-python

# command to install on a GPU
# !CMAKE_ARGS="-DGGML_CUDA=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 MB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00


In [10]:
from langchain_community.llms import LlamaCpp
from langchain import PromptTemplate
# <|user|>\n{prompt}<|end|>\n<|assistant|>
template = """<|user|>
 {input_prompt}
 <|end|>
 <|assistant|>"""

prompt = PromptTemplate.from_template(template)

# Make sure the model path is correct for your system!
llm = LlamaCpp(
  model_path="/content/Phi-3-mini-4k-instruct-q4.gguf",
  # n_gpu_layers=-1,
  stop=["<|end|>"],
  max_tokens=1000,
  n_ctx=2048,
  seed=42,
)

basic_chain = prompt | llm

question = """"
"input_prompt": "Create a sci-fi story summary",
"""
basic_chain.invoke(question)

llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from /content/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count_kv u32  

' Title: Echoes of the Lost Nebula\n\nIn the year 2450, humanity has ventured far beyond the confines of Earth and colonized numerous planets within the Andromeda galaxy. However, a long-forgotten ancient alien civilization known as The Luminae left behind an enigmatic nebula that holds unimaginable powers and secrets.\n\nAs Earth\'s largest interstellar corporation, Helix Infinity Systems (HIS) leads the expedition to explore this mysterious Nebula of Echoes. A team comprised of renowned scientists, engineers, astronauts, and a highly-skilled special forces operative named Commander Eli Vega is assembled for the mission: unraveling the secrets held within the nebula while protecting HIS interests from rival corporations seeking to exploit its power.\n\nHowever, as they venture deeper into the Echo Nebula, strange occurrences begin plaguing their crew and technology. The team discovers that certain individuals can tap into psychic abilities once exposed to radiation within the nebula. 

In [11]:
from langchain import LLMChain

# Create a chain for the title of our story
template = """<|user|>
Create a title for a story about {summary}. Only return the title.
<|end|>
<|assistant|>"""
title_prompt = PromptTemplate(template=template, input_variables=["summary"])
title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")

title.invoke({"summary": """In the year 2450, humanity has ventured far beyond the confines of Earth and colonized numerous planets within the Andromeda galaxy. However, a long-forgotten ancient alien civilization known as The Luminae left behind an enigmatic nebula that holds unimaginable powers and secrets.\n\nAs Earth\'s largest interstellar corporation, Helix Infinity Systems (HIS) leads the expedition to explore this mysterious Nebula of Echoes. A team comprised of renowned scientists, engineers, astronauts, and a highly-skilled special forces operative named Commander Eli Vega is assembled for the mission: unraveling the secrets held within the nebula while protecting HIS interests from rival corporations seeking to exploit its power.\n\nHowever, as they venture deeper into the Echo Nebula, strange occurrences begin plaguing their crew and technology. The team discovers that certain individuals can tap into psychic abilities once exposed to radiation within the nebula."""})

<ipython-input-11-f662d52b980f>:9: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  title = LLMChain(llm=llm, prompt=title_prompt, output_key="title")
Llama.generate: 2 prefix-match hit, remaining 250 prompt tokens to eval
llama_perf_context_print:        load time =    7113.65 ms
llama_perf_context_print: prompt eval time =   67512.82 ms /   250 tokens (  270.05 ms per token,     3.70 tokens per second)
llama_perf_context_print:        eval time =    8479.92 ms /    20 runs   (  424.00 ms per token,     2.36 tokens per second)
llama_perf_context_print:       total time =   76020.38 ms /   270 tokens


{'summary': "In the year 2450, humanity has ventured far beyond the confines of Earth and colonized numerous planets within the Andromeda galaxy. However, a long-forgotten ancient alien civilization known as The Luminae left behind an enigmatic nebula that holds unimaginable powers and secrets.\n\nAs Earth's largest interstellar corporation, Helix Infinity Systems (HIS) leads the expedition to explore this mysterious Nebula of Echoes. A team comprised of renowned scientists, engineers, astronauts, and a highly-skilled special forces operative named Commander Eli Vega is assembled for the mission: unraveling the secrets held within the nebula while protecting HIS interests from rival corporations seeking to exploit its power.\n\nHowever, as they venture deeper into the Echo Nebula, strange occurrences begin plaguing their crew and technology. The team discovers that certain individuals can tap into psychic abilities once exposed to radiation within the nebula.",
 'title': ' "Echoes of t

In [ ]:
# Create a chain for the character description using the summary and title
template = """<s><|user|>
Describe the main character of a story about {summary} with the title
{title}. Use only two sentences.
<|end|>
<|assistant|>"""
character_prompt = PromptTemplate(template=template, input_variables=["summary", "title"]
)
character = LLMChain(llm=llm, prompt=character_prompt, output_key="character")